In [25]:
import cv2
import time
import FaceNormalizationUtils as faceutils
import pickle
import numpy as np
import os
from scipy import stats
# mode and so on
from collections import Counter

#MTCNN face detector
from mtcnn.mtcnn import MTCNN
#deepface
from deepface import DeepFace
from deepface.commons import functions

from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.neighbors import NearestNeighbors

In [26]:
def calc_embs(X, batch_size=2):
    norm_images = prewhiten(X)
    pd = []
    for start in range(0, len(norm_images), batch_size):
        pd.append(model.predict_on_batch(norm_images[start:start+batch_size]))
    return l2_normalize(np.concatenate(pd))

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

# Designed for UTKFace as filename contains biometric information
def TrainEmbedingsUTKFace(folders,outputfile):
    nimgs = 0
    Xorig = []
    X = []
    Gender = []
    Age = []
    Etnia = []
    
    for directory in folders:
        #print(directory)
        for path, subdirs, files in os.walk(directory):
            for name in files:
                #print(name)
                if name.endswith(".jpg") and nimgs < 2000:
                #if name.endswith(".png"): # and nimgs < 3000:
                    img_path = os.path.join(path, name)
                    #print(img_path)

                    image = cv2.imread(img_path)

                    if (type(image) is np.ndarray):
                        #print(img_path)
                        # Search face 
                        values = DetectLargestFaceEyesMTCNN(image)
                        if values is not None:
                            #print(nimgs)
                            face, eyes, shape = values

                            # draws container
                            [x, y, w, h] = face
                            if x > -1:
                                # Eyes
                                [lex, ley, rex, rey] = eyes
                                if lex > -1:
                                    
                                    B, G, R = cv2.split(image)

                                    # Normalize for Facenet
                                    normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Bnorm = normalizatorHS.normf_image
                                    normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Gnorm = normalizatorHS.normf_image
                                    normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                                    Rnorm = normalizatorHS.normf_image
                                    NormBGRHS = cv2.merge((Bnorm, Gnorm, Rnorm))
                                    #cv2.imshow("Normalized", NormBGRHS)

                                    # Cropping from HS for facenet
                                    # Usa nyoki https://github.com/nyoki-mtl/keras-facenet
                                    NormBGR = NormBGRHS[35:115, 39:119, :]

                                    # SOft biometric data are extracted from filename
                                    if NormBGR is not None:
                                        nimgs = nimgs + 1
                                        #print(nimgs)

                                        fsub = name.find('_')
                                        sage = name[:fsub]
                                        #print(sage)
                                        sub = name[fsub + 1:]
                                        fsub = sub.find('_')
                                        sgender = sub[:fsub]
                                        sub = sub[fsub + 1:]
                                        fsub = sub.find('_')
                                        setnia = sub[:fsub]

                                        # print(name)
                                        # print(sage)
                                        # print(sgender)
                                        # print(setnia)
                                        # print(nimgs)

                                        # Facenet kearas expects 160x160
                                        #imaged = cv2.resize(NormBGR, (160, 160))
                                        
                                        # Obtiene embeddings
                                        imaged = cv2.resize(NormBGR, dim, interpolation = cv2.INTER_AREA)
                                        
                                        # Mantengo originales para mostrar parecido
                                        imager = cv2.resize(image, (200, 200), interpolation=cv2.INTER_AREA)
                                        Xorig.append(imager)
                                        # Facenet
                                        X.append(imaged)
                                        Gender.append(sgender)
                                        Age.append(sage)
                                        Etnia.append(setnia)
    
    if nimgs > 0:
        # Compute embeddings 
        embs = calc_embs(np.array(X))

        fid = open(outputfile, "wb")

        pickle.dump([nimgs, X, embs, Age, Gender, Etnia], fid)
        fid.close()


In [27]:
def GetSimilarNN(X,idxsimilar, nbrs, embs, Title):
    # Gets nearest neighbors
    distance, indices = nbrs.kneighbors(embs)
    minp = indices[0][0]

    # Keeps copy of last grames NN
    idxsimilar.extend(indices[0])

    # Remove old ones when more than nframes accumulated
    if len(idxsimilar) > kvecinos * nframeskvecinos:
        idxsimilar = idxsimilar[kvecinos:]

    # print('Lista con ' + str(len(idxsimilar)) + ' ' + str(idxsimilar) + '\n')

    # enough history
    if len(idxsimilar) > kvecinos:
        minp = GetClosesetMode(idxsimilar)

    # Larger for visualization
    imageS = cv2.resize(X[minp], (320, 320))

    # Soft biometrics labels
    if len(idxsimilar) > 0:
        print(idxsimilar)
        # print([int(Gender[int(i)]) for i in idxsimilar], [int(Etnia[int(i)]) for i in idxsimilar])
        genderAux = []
        etniaAux = []
        for i in idxsimilar:
            try:
                etniaAux.append(int(Etnia[int(i)]))
                genderAux.append(int(Gender[int(i)]))
            except Exception as e:
                complex
        modesG = stats.mode(genderAux)
        modesE = stats.mode(etniaAux)
        edad = np.array([int(Age[int(i)]) for i in idxsimilar]).mean()

        if modesG[0] == 0:
            gen = 'M'
        else:
            gen = 'F'

        if modesE[0] == 0:
            et = 'B'
        elif modesE[0] == 1:
            et = 'N'
        elif modesE[0] == 2:
            et = 'A'
        elif modesE[0] == 3:
            et = 'H'
        else:
            et = 'L'

        cv2.putText(imageS, '%s %s %d (%s)' % (gen, et, int(edad), Age[minp]), (10, 30), font, 0.5, (255, 255, 255), 2,
                    cv2.LINE_AA)

    cv2.imshow(Title, imageS)

    return idxsimilar

def getLargestMTCNNBB(objects):
        if len(objects) < 1:
            return -1
        elif len(objects) == 1:
            return 0
        else:
            areas = [ (det['box'][2]*det['box'][3]) for det in objects ]
            return np.argmax(areas)
        
def DetectLargestFaceEyesMTCNN(img):
    global detectormtcnn
    
    results = detectormtcnn.detect_faces(img)

    if not results is None:
        index = getLargestMTCNNBB(results)

        if len(results) < 1:
            return None

        # laergest face
        face_info = results[index]

        #print(face_info)

        [x, y, w, h] = face_info['box']
        le = face_info['keypoints']['left_eye']
        re = face_info['keypoints']['right_eye']

        return [x,y,w,h], [le[0], le[1], re[0], re[1]], [face_info['keypoints']['left_eye'], face_info['keypoints']['right_eye'],
                      face_info['keypoints']['nose'], face_info['keypoints']['mouth_left'],
                      face_info['keypoints']['mouth_right']]
    else:
        return None
    
def ResetDetectionCounters():
    global nconsecutivenodetected, nconsecutivedetected, idxsimilarFN

    nconsecutivenodetected = nconsecutivenodetected + 1
    if nconsecutivenodetected > 10:
        nconsecutivedetected = 0
        idxsimilarFN = []

def GetClosesetMode(list):
    # Get occurrences list
    occ = Counter(list)

    # Get the mode, if multiple modes present, gets the clostest observing position among beigbors
    prima = 0
    maxpun = 0
    for neighbor, count in occ.most_common(10):
        if prima == 0:
            prima = 1
            mode = count
        else:
            if count < mode:
                break

        #print('%s: %7d' % (neighbor, count))

        # using enumerate()
        # to find indices for 3
        neighbor_pos = [i for i, value in enumerate(list) if value == neighbor]
        # printing resultant list
        #print("New indices list : " + str(neighbor_pos))

        pun = 0
        for pos in neighbor_pos:
            pun = pun + kvecinos - (pos % kvecinos)
        #print(pun)
        if pun > maxpun:
            maxpun = pun
            closest_neighbor = neighbor

    return closest_neighbor

def changeSign(lst):
    return [-i for i in lst]

In [28]:
# Face detector
detectormtcnn = MTCNN()
# Normalization utilities
normalizatorHS = faceutils.Normalization()

# Embeddings deepface
model = DeepFace.build_model("Facenet")
target_size = model.layers[0].input_shape
dim = (int(target_size[0][1]), int(target_size[0][2]))

# 1 to create a new appearance dataset
creadataset = 0
if creadataset == 1:
    print("Create model")
    #Dataset folders
    folders = ["Datasets/part1/part1", "Datasets/part2/part2",
               "Datasets/part3/part3"]
    # TrainEmbedingsUTKFace(folders,"D:/FACEScode_models/UTKFace_DLIB_Nyoki_deepfaceSP2021.obj")

#Load appearance dataset
print("Reading model")
#MODIFICAR EN BASE A TU RUTA
fid = open("UTKFace_DLIB_Nyoki.obj", "rb") #Modelo reducido
#fid = open("E:/FACEScode_models/UTKFace_DLIB_Nyoki.obj", "rb") # Entrenado similar a normalización Nyoki con 20k
#fid = open("D:/FACEScode_models/UTKFace_DLIB_Nyoki_deepfaceSP2021.obj", "rb") # Entrenado similar a normalización Nyoki con 2k
nimgs, X_FN, embsFN, Age, Gender, Etnia = pickle.load(fid)
#print(embsFN.shape)

# Tree for KNN search
kvecinos = 10
nframeskvecinos = 5
nbrsFN = NearestNeighbors(n_neighbors=kvecinos).fit(embsFN)

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection, check unitl one is located
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(2)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 2')
    else:
        print('Camera 1')
else:
    print('Camera 2')
    
#Set camera resolution
cap.set(3,640);
cap.set(4,480);

# Initializations
debug = 0
nconsecutivedetected = 0
nconsecutivenodetected = 0
idxsimilarFN = []

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization split channels
    B, G, R = cv2.split(frame)

    # Search face 
    values = DetectLargestFaceEyesMTCNN(frame)
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:                
                nconsecutivedetected = nconsecutivedetected + 1  #11?
                nconsecutivenodetected = 0
                    
                # Show detected facial elements
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)

                # HS normalization for facenet
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGRHS = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGRHS)
                    
                # Cropping from HS for facenet
                # De HS a lo proximadamente usa nyok https://github.com/nyoki-mtl/keras-facenet
                NormBGR = NormBGRHS[35:115, 39:119, :]
                cv2.imshow("Normalizedc", NormBGR)
                 
                # Obtiene embeddings
                img1 = cv2.resize(NormBGR, dim, interpolation = cv2.INTER_AREA)
                #embs = model.predict(img1[None,...])
                embs = calc_embs(np.array([img1]))
                embs = changeSign(embs)
                idxsimilarFN = GetSimilarNN(X_FN,idxsimilarFN, nbrsFN, embs, 'FN Me recuerdas a ...')
                
            else:
                ResetDetectionCounters()
        else:
            ResetDetectionCounters()
    else:
        ResetDetectionCounters()

    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(5)
    if tec & tec == 27:  # Esc
        break  

# Close windows and release camera
cap.release()
cv2.destroyAllWindows()

Reading model
Camera 2
1/1 [==============================] - 0s 118ms/step
[13588, 635, 17831, 15729, 20458, 21433, 21391, 4, 11007, 11220]
1/1 [==============================] - 0s 29ms/step
[13588, 635, 17831, 15729, 20458, 21433, 21391, 4, 11007, 11220, 4766, 4, 21771, 4184, 15633, 21391, 5538, 21203, 19404, 18634]
1/1 [==============================] - 0s 21ms/step
[13588, 635, 17831, 15729, 20458, 21433, 21391, 4, 11007, 11220, 4766, 4, 21771, 4184, 15633, 21391, 5538, 21203, 19404, 18634, 4, 21771, 4184, 21203, 4766, 19404, 18634, 5538, 21391, 15633]
1/1 [==============================] - 0s 22ms/step
[13588, 635, 17831, 15729, 20458, 21433, 21391, 4, 11007, 11220, 4766, 4, 21771, 4184, 15633, 21391, 5538, 21203, 19404, 18634, 4, 21771, 4184, 21203, 4766, 19404, 18634, 5538, 21391, 15633, 15633, 4, 4834, 21771, 21391, 18634, 19106, 4766, 20928, 4184]
1/1 [==============================] - 0s 21ms/step
[13588, 635, 17831, 15729, 20458, 21433, 21391, 4, 11007, 11220, 4766, 4, 2177